In [ ]:
import cudf
import rmm
import gc
import Path

In [ ]:
OUTPUT_DIR = Path("~/Projects/proyecto-computacion-I/data/datasets/processed/measurements.parquet").expanduser()

In [ ]:
rmm.reinitialize(managed_memory=True)

In [23]:
# 1. Leer el parquet
df_gpu = cudf.read_parquet(OUTPUT_DIR)

# Ahora ya puedes hacer tus análisis
print(df_gpu.info())
print(df_gpu.head())

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 899765 entries, 0 to 899764
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   timestamp  899765 non-null  datetime64[ns]
 1   value      899765 non-null  float64
 2   sensor_id  899765 non-null  object
 3   metric     899765 non-null  object
 4   unit       899765 non-null  object
 5   batch      899765 non-null  category
dtypes: category(1), datetime64[ns](1), float64(1), object(3)
memory usage: 39.8+ MB
None
            timestamp  value sensor_id            metric  unit   batch
0 2025-10-02 14:00:00    7.4      C323                PH    ph  batch1
1 2025-10-02 14:00:00    0.1      C322            AMONIO   ppm  batch1
2 2025-10-02 14:00:00    2.2      C342         CLOROFILA  µg/l  batch1
3 2025-10-02 14:00:00    1.9      C323  CARBONO ORGANICO   ppm  batch1
4 2025-10-02 14:00:00   12.6      C313         CLOROFILA   ppb  batch1


El dataframe combinado es el dataframe sin el marcador del batch

In [24]:
df = df_gpu.drop(columns=['batch'])

In [26]:
print(f"Filas originales: {len(df_gpu)}")

# 2. El modo DIOS: Multiplicar los datos x 2500
# Concatenamos el dataframe consigo mismo 2500 veces
print("Inflando datos (x1000)...")
df_big = cudf.concat([df_gpu] * 1000, ignore_index=True)

print(f"Nuevas filas: {len(df_big):,}") # Deberían ser ~156 millones
print(f"Uso de memoria estimado: {df_big.memory_usage(deep=True).sum() / 1024**3:.2f} GB")

# 3. Operación pesada: Rolling Window compleja
# Calculamos la media móvil de 7 días sobre 156 millones de filas
print("Ejecutando cálculo pesado...")
%time res = df_big['value'].rolling(window=168).mean() # 168 horas = 7 días

print("¡Cálculo terminado!")

Filas originales: 899765
Inflando datos (x2500)...


OverflowError: CUDF failure at: /tmp/conda-bld-output/bld/rattler-build_libcudf/work/cpp/src/copying/concatenate.cu:478: Total number of concatenated rows exceeds the column size limit

In [27]:
del df_big

NameError: name 'df_big' is not defined

In [ ]:
gc.collect()

print("Memoria liberada (Lógica). RMM la tiene lista para reusar.")